# RFC 001: Global run function

In [ ]:
from abc import ABC, abstractmethod
from typing import Annotated, Any, Iterable, Literal, Optional, Protocol
from unittest.mock import MagicMock
from uuid import UUID, uuid4

from pydantic import BaseModel, Field

from autogen import Agent

## Messages

Eventually, we will move away from dictionaries and represent all messages with objects (dataclasses or Pydantic base models). For now, we will use existing implementation.

In [ ]:
Message = dict[str, Any]

## Events

These are six different abstract types of events used by the runtime.

In [ ]:
EventType = Literal["input_request", "async_input_request", "input_response", "agent_message", "output", "system"]


class Event(BaseModel, ABC):
    uuid: Annotated[UUID, Field(default_factory=uuid4)]

    type: EventType

    @abstractmethod
    def process(self, event_processor: "EventProcessorProtocol"): ...

    @abstractmethod
    async def a_process(self, event_processor: "AsyncEventProcessorProtocol"): ...


class InputRequestEvent(Event):
    prompt: str

    def respond(self, response: "InputResponseEvent"):
        pass

    type: EventType = "input_request"

    def process(self, event_processor: "EventProcessorProtocol"):
        print(f"InputRequestEvent.process: {self=}, {event_processor=}")
        event_processor.process_input_request_event(self)

    async def a_process(self, event_processor: "AsyncEventProcessorProtocol"):
        await event_processor.a_process_input_request_event(self)


class AsyncInputRequestEvent(Event):
    prompt: str

    async def a_respond(self, response: "InputResponseEvent"):
        pass

    type: EventType = "async_input_request"

    def process(self, event_processor: "EventProcessorProtocol"):
        raise RuntimeError("Cannot process async input request synchronously")

    async def a_process(self, event_processor: "AsyncEventProcessorProtocol"):
        await event_processor.a_process_async_input_request_event(self)


class InputResponseEvent(Event):
    type: EventType = "input_response"

    value: str

    def process(self, event_processor: "EventProcessorProtocol"):
        event_processor.process_input_response_event(self)

    async def a_process(self, event_processor: "AsyncEventProcessorProtocol"):
        await event_processor.a_process_input_response_event(self)


class AgentMessageEvent(Event):
    message: Message

    type: EventType = "agent_message"

    def process(self, event_processor: "EventProcessorProtocol"):
        event_processor.process_agent_message_event(self)

    async def a_process(self, event_processor: "AsyncEventProcessorProtocol"):
        await event_processor.a_process_agent_message_event(self)


class OutputEvent(Event):
    value: str

    type: EventType = "output"

    def process(self, event_processor: "EventProcessorProtocol"):
        event_processor.process_output_event(self)

    async def a_process(self, event_processor: "AsyncEventProcessorProtocol"):
        await event_processor.a_process_output_event(self)


class SystemEvent(Event):
    value: str

    type: EventType = "system"

    def process(self, event_processor: "EventProcessorProtocol"):
        event_processor.process_system_event(self)

    async def a_process(self, event_processor: "AsyncEventProcessorProtocol"):
        await event_processor.a_process_system_event(self)

Helpers

In [ ]:
def is_input_request(event: Event) -> bool:
    return event.type == "input_request"


def is_async_input_request(event: Event) -> bool:
    return event.type == "async_input_request"


def is_input_response(event: Event) -> bool:
    return event.type == "input_response"


def is_output(event: Event) -> bool:
    return event.type == "output"


def is_agent_message(event: Event) -> bool:
    return event.type == "agent_message"


def is_system_event(event: Event) -> bool:
    return event.type == "system"

## Global run function

In [ ]:
from typing import AsyncIterable


class RunResponseProtocol(Protocol):
    @property
    def events(self) -> Iterable[Event]: ...

    @property
    def messages(self) -> Iterable[Message]: ...

    @property
    def summary(self) -> str: ...


def process_run_response(run_response: RunResponseProtocol, processor: "EventProcessorProtocol") -> None:
    for event in run_response.events:
        event.process(processor)


class AsyncRunResponseProtocol(Protocol):
    @property
    def events(self) -> AsyncIterable[Event]: ...

    @property
    def messages(self) -> AsyncIterable[Message]: ...

    @property
    async def summary(self) -> str: ...

In [ ]:
def run(
    *agents: Agent, message: Optional[str] = None, previous_run: Optional[RunResponseProtocol] = None, **kwargs: Any
) -> RunResponseProtocol:
    """Run the agents with the given initial message.

    Args:
        agents: The agents to run.
        message: The initial message to send to the first agent.
        previous_run: The previous run to continue.
        kwargs: Additional arguments to pass to the agents.

    """
    ...


async def a_run(
    *agents: Agent, message: Optional[str] = None, previous_run: Optional[RunResponseProtocol] = None, **kwargs: Any
) -> AsyncRunResponseProtocol:
    """Run the agents with the given initial message.

    Args:
        agents: The agents to run.
        message: The initial message to send to the first agent.
        previous_run: The previous run to continue.
        kwargs: Additional arguments to pass to the agents.

    """
    ...

## Event processing loop

In [ ]:
import unittest

from autogen import ConversableAgent

agents: list[Agent] = [ConversableAgent(name="Alice"), ConversableAgent(name="Bob")]

response = run(*agents, message="What is the meaning of life?")

# mocking response
response = MagicMock(spec=RunResponseProtocol)
response.events = [
    InputRequestEvent(prompt="What is the meaning of life?"),
    InputResponseEvent(value="42"),
    AgentMessageEvent(message={"text": "What is the meaning of life?", "sender": "Alice"}),
    OutputEvent(value="thinking about it..."),
    SystemEvent(value="done"),
]
response.messages = [
    {"text": "What is the meaning of life?", "sender": "Alice"},
]
response.summary = "Alice and Bob had a conversation about the meaning of life."

with unittest.mock.patch("builtins.input", return_value="42"):
    for event in response.events:
        print(f"Event: {event}")
        if is_input_request(event):
            print(f"Input request: {event.prompt}")
            s = input(event.prompt)

            print(f"Response from UI: {s}")
            event.respond(s)

        elif is_output(event):
            print(f"Output message: {event.value}")

        elif is_system_event(event):
            print(f"System event: {event}")

print()
print(f"Messages: {response.messages}")

print()
print(f"Summary: {response.summary}")

## Event processor classes

In [ ]:
class EventProcessorProtocol(Protocol):
    def process_input_request_event(self, event: Event): ...

    def process_input_response_event(self, event: Event): ...

    def process_agent_message_event(self, event: Event): ...

    def process_output_event(self, event: Event): ...

    def process_system_event(self, event: Event): ...


class SimpleEventProcessor:
    def process_input_request_event(self, event: Event):
        print(f"Input request: {event.prompt}")
        s = input(event.prompt)

        print(f"Response from UI: {s}")
        event.respond(s)

    def process_input_response_event(self, event: Event):
        print(f"Input response: {event.value}")

    def process_agent_message_event(self, event: Event):
        print(f"Agent message: {event.message}")

    def process_output_event(self, event: Event):
        print(f"Output message: {event.value}")

    def process_system_event(self, event: Event):
        print(f"System event: {event}")

In [ ]:
with unittest.mock.patch("builtins.input", return_value="42"):
    process_run_response(response, SimpleEventProcessor())

## Examples runs

### Single agent

### Two agents

### Group chat

### Swarm